# Training a MLP using backpropagation

In the following tutorial we look at the main features of **VINA**. In particular, we see how to define different classes of models, with an easy and user friendly interface.

* **part 1** we train using `MLP`. 
* **part 2** we train using `Sequential` and `Linear`. 
* **part 3** we train using `Sequential`, `Linear` and `Conv2D`. 

In [9]:
import sys

sys.path.append('../')

import jax
from jax import numpy as jnp
from jax import grad, jit
from veni import ReLU, LogSoftmax, MLP, Module, Sequential, Linear, Conv2D, MaxPool2D, Flatten, Softmax
from time import time

jax.config.update('jax_platform_name', 'cpu')

## Part 1
We first define useful parameters that we are going to use later. Note that our model is a MLP imported from `net.py`

In [10]:
# hyperparameters
layer_sizes = [784, 512, 256, 10]
step_size = 0.01
num_epochs = 5
batch_size = 128
n_targets = 10
key = jax.random.PRNGKey(0)
function = ReLU()
log_softmax = LogSoftmax()

# to create a model is as simple as that
model = MLP(layer_sizes, function, key)
params = model.params # we need to extract parameters

Helper functions, trainloader and other stuff...

In [11]:
# some helper functions
def one_hot(x, k, dtype=jnp.float32):
    """Create a one-hot encoding of x of size k."""
    return jnp.array(x[:, None] == jnp.arange(k), dtype)
  
def accuracy(params, images, targets):
    target_class = jnp.argmax(targets, axis=1)
    predicted_class = jnp.argmax(model(images,params), axis=1)
    return jnp.mean(predicted_class == target_class)

def loss(params, images, targets):
    preds = model(images, params)
    preds = log_softmax(preds)
    return -jnp.mean(preds * targets)

@jit
def update(params, x, y):
    grads = grad(loss)(params, x, y)
    return [(w - step_size * dw, b - step_size * db)
          for (w, b), (dw, db) in zip(params, grads)]


import numpy as np
from torch.utils import data
from torchvision.datasets import MNIST

def numpy_collate(batch):
    if isinstance(batch[0], np.ndarray):
        return np.stack(batch)
    elif isinstance(batch[0], (tuple,list)):
        transposed = zip(*batch)
        return [numpy_collate(samples) for samples in transposed]
    else:
        return np.array(batch)

class NumpyLoader(data.DataLoader):
    def __init__(self, dataset, batch_size=1,
                shuffle=False, sampler=None,
                batch_sampler=None, num_workers=0,
                pin_memory=False, drop_last=False,
                timeout=0, worker_init_fn=None):
        super(self.__class__, self).__init__(dataset,
            batch_size=batch_size,
            shuffle=shuffle,
            sampler=sampler,
            batch_sampler=batch_sampler,
            num_workers=num_workers,
            collate_fn=numpy_collate,
            pin_memory=pin_memory,
            drop_last=drop_last,
            timeout=timeout,
            worker_init_fn=worker_init_fn)

class FlattenAndCast(object):
    def __call__(self, pic):
        return np.ravel(np.array(pic, dtype=jnp.float32))

# Define our dataset, using torch datasets
mnist_dataset = MNIST('/tmp/mnist/', download=True, transform=FlattenAndCast())
training_generator = NumpyLoader(mnist_dataset, batch_size=batch_size, num_workers=0)

# Get the full train dataset (for checking accuracy while training)
train_images = np.array(mnist_dataset.train_data).reshape(len(mnist_dataset.train_data), -1)
train_labels = one_hot(np.array(mnist_dataset.train_labels), n_targets)

# Get full test dataset
mnist_dataset_test = MNIST('/tmp/mnist/', download=True, train=False)
test_images = jnp.array(mnist_dataset_test.test_data.numpy().reshape(len(mnist_dataset_test.test_data), -1), dtype=jnp.float32)
test_labels = one_hot(np.array(mnist_dataset_test.test_labels), n_targets)

/usr/local/lib/python3.9/site-packages/torchvision/datasets/mnist.py:75: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.9/site-packages/torchvision/datasets/mnist.py:65: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.9/site-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.9/site-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Here we train using backpropagation

In [13]:
for epoch in range(num_epochs):
    start_time = time()
    for x, y in training_generator:
        y = one_hot(y, n_targets)
        params = update(params, x, y)
        epoch_time = time() - start_time

    train_acc = accuracy(params, train_images, train_labels)
    test_acc = accuracy(params, test_images, test_labels)
    print("Epoch {} in {:0.2f} sec".format(epoch, epoch_time))
    print("Training set accuracy {}".format(train_acc))
    print("Test set accuracy {}".format(test_acc))

Epoch 0 in 3.49 sec
Training set accuracy 0.9045166373252869
Test set accuracy 0.9120000004768372
Epoch 1 in 3.04 sec
Training set accuracy 0.9275500178337097
Test set accuracy 0.9304999709129333
Epoch 2 in 3.03 sec
Training set accuracy 0.9414166808128357
Test set accuracy 0.9398999810218811
Epoch 3 in 3.18 sec
Training set accuracy 0.9497833251953125
Test set accuracy 0.9477999806404114
Epoch 4 in 3.17 sec
Training set accuracy 0.956416666507721
Test set accuracy 0.9527999758720398


## Part 2
We first define useful parameters that we are going to use later. Note that our model is a costum model built using `Sequential` and `Linear`.

In [40]:
class myModel(Module):
    def __init__(self):
        self.layers = Sequential([
            Linear(28*28,512,key),
            function,
            Linear(512,256,key),
            function,
            Linear(256,10,key)
        ])
        self.params = self.layers.generate_parameters()

    def forward(self, data, params):
        return self.layers(data, params)


# we define the model as simple as that
model = myModel()
params = model.params

Let's train

In [6]:
for epoch in range(num_epochs):
    start_time = time.time()
    for x, y in training_generator:
        y = one_hot(y, n_targets)
        params = update(params, x, y)
        epoch_time = time.time() - start_time

  
    train_acc = accuracy(params, train_images, train_labels)
    test_acc = accuracy(params, test_images, test_labels)
    print("Epoch {} in {:0.2f} sec".format(epoch, epoch_time))
    print("Training set accuracy {}".format(train_acc))
    print("Test set accuracy {}".format(test_acc))

Epoch 0 in 6.04 sec
Training set accuracy 0.9094333648681641
Test set accuracy 0.9138000011444092
Epoch 1 in 5.02 sec
Training set accuracy 0.9317333698272705
Test set accuracy 0.9314999580383301
Epoch 2 in 4.85 sec
Training set accuracy 0.9435499906539917
Test set accuracy 0.9408999681472778


## Part 3
We first define useful parameters that we are going to use later. Note that our model is a costum model built using `Sequential`, `Linear` and `Conv2D` base on this [PyTorch tutorial](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html).

In [6]:
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms

# hyperparameters
step_size = 0.01
num_epochs = 5
batch_size = 4
n_targets = 10
key = jax.random.PRNGKey(0)
function = ReLU()
log_softmax = LogSoftmax()

class Cast(object):
    def __call__(self, pic):
        return np.array(pic, dtype=jnp.float32)

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), Cast()])

cifar_dataset = CIFAR10('/tmp/cifar/', download=True,train=True,
                        transform=transform)
training_generator = NumpyLoader(
    cifar_dataset, batch_size=batch_size, num_workers=0)

# Get full test dataset
cifar_dataset_test = CIFAR10('/tmp/cifar/', download=True, train=False,
                             transform=Cast())
testing_generator = NumpyLoader(
    cifar_dataset, batch_size=batch_size, num_workers=0)


Files already downloaded and verified
Files already downloaded and verified


Let's define the model

In [7]:
class CNNModel(Module):
    def __init__(self):
        self.layers = Sequential([
            Conv2D(3, 6, 5, 1, 'VALID', jax.random.PRNGKey(0)),
            function,
            MaxPool2D(2, 2),
            Conv2D(6, 16, 5, 1, 'VALID', jax.random.PRNGKey(1)),
            function,
            MaxPool2D(2, 2),
            Flatten(),
            Linear(16 * 5 * 5, 120, jax.random.PRNGKey(2)),
            function,
            Linear(120, 84, jax.random.PRNGKey(3)),
            function,
            Linear(84, 10, jax.random.PRNGKey(3))
        ])
        self.params = self.layers.generate_parameters()

    def forward(self, data, params):
        return self.layers(data, params)


# we define the model as simple as that
model = CNNModel()
params = model.params

We now train our model

In [8]:
for epoch in range(num_epochs):
    running_loss = 0
    for i, (image, label) in enumerate(training_generator):
        one_hot_label = one_hot(label, n_targets)
        params = update(params, image, one_hot_label)
        
        # loss info
        loss_item =  loss(params, image, one_hot_label)
        running_loss = running_loss + loss_item
        if i % 2000 == 1999:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0  

[1,  2000] loss: 0.216
[1,  4000] loss: 0.186
[1,  6000] loss: 0.165
[1,  8000] loss: 0.153
[1, 10000] loss: 0.152
[1, 12000] loss: 0.145
[2,  2000] loss: 0.137
[2,  4000] loss: 0.138
[2,  6000] loss: 0.132
[2,  8000] loss: 0.129
[2, 10000] loss: 0.132
[2, 12000] loss: 0.128


We now validate the model

In [28]:
correct = 0
total = 0
for data in testing_generator:
    images, labels = data
    output = model(images,params)
    predicted = jnp.argmax(Softmax()(output), 1)
    total += labels.shape[0]
    correct += float((predicted == labels).sum())
    
print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')
    

Accuracy of the network on the 10000 test images: 42.0 %


In [31]:
# prepare to count predictions for each class
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

for data in testing_generator:
    images, labels = data
    output = model(images,params)
    predicted = jnp.argmax(Softmax()(output), 1)
    for label, prediction in zip(labels, predicted):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1
            
# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy for class: plane is 47.0 %
Accuracy for class: car   is 77.6 %
Accuracy for class: bird  is 21.1 %
Accuracy for class: cat   is 24.7 %
Accuracy for class: deer  is 38.2 %
Accuracy for class: dog   is 16.6 %
Accuracy for class: frog  is 50.3 %
Accuracy for class: horse is 58.9 %
Accuracy for class: ship  is 62.0 %
Accuracy for class: truck is 29.5 %
